In [5]:
import cv2
import os
from PIL import Image
import numpy as np
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from keras.utils import normalize

from keras.models import Sequential
from keras.layers import Conv2D, MaxPool2D, Activation, Dropout, Flatten, Dense


In [24]:
image_directory= r"D:\abcdefghj\Projects\Covid_Detection\Brain_Tumor\\"
no_tumor_images=os.listdir(image_directory+"/no")
yes_tumor_images=os.listdir(image_directory+"/yes")
dataset=[]
label=[]

# no_tumor_images
for i, image_name in enumerate(no_tumor_images):
    if(image_name.split('.')[1]=='jpg'):
        image=cv2.imread(image_directory+'no//'+image_name)
        image=Image.fromarray(image, 'RGB')
        image=image.resize((64,64))
        dataset.append(np.array(image))
        label.append(0)
        
for i, image_name in enumerate(yes_tumor_images):
    if(image_name.split('.')[1]=='jpg'):
        image=cv2.imread(image_directory+'yes//'+image_name)
        image=Image.fromarray(image, 'RGB')
        image=image.resize((64,64))
        dataset.append(np.array(image))
        label.append(1)
        
# print(dataset)

In [29]:
dataset=np.array(dataset)
label=np.array(label)

In [49]:
x_train, x_test, y_train, y_test=train_test_split(dataset, label, test_size=0.2, random_state=0)

print(x_train.shape)

(2400, 64, 64, 3)


In [50]:
x_train= normalize(x_train, axis=1)
x_test= normalize(x_test, axis=1)

In [51]:
model= Sequential()

model.add(Conv2D(32,(3,3), input_shape=(64,64,3)))
model.add(Activation('relu'))
model.add(MaxPool2D(pool_size=(2,2)))

model.add(Conv2D(32,(3,3), kernel_initializer='he_uniform'))
model.add(Activation('relu'))
model.add(MaxPool2D(pool_size=(2,2)))

model.add(Conv2D(32,(3,3), kernel_initializer='he_uniform'))
model.add(Activation('relu'))
model.add(MaxPool2D(pool_size=(2,2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

In [52]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [53]:
model.fit(x_train, y_train, batch_size=16, verbose=1, epochs=10, validation_data=(x_test, y_test) ,shuffle=False)

Epoch 1/10
150/150 [==============================] - 6s 38ms/step - loss: 0.5463 - accuracy: 0.7179 - val_loss: 0.4277 - val_accuracy: 0.8067
Epoch 2/10
150/150 [==============================] - 6s 38ms/step - loss: 0.3965 - accuracy: 0.8317 - val_loss: 0.3699 - val_accuracy: 0.8383
Epoch 3/10
150/150 [==============================] - 5s 35ms/step - loss: 0.3143 - accuracy: 0.8712 - val_loss: 0.2857 - val_accuracy: 0.8617
Epoch 4/10
150/150 [==============================] - 6s 37ms/step - loss: 0.2531 - accuracy: 0.9004 - val_loss: 0.2456 - val_accuracy: 0.9000
Epoch 5/10
150/150 [==============================] - 5s 36ms/step - loss: 0.1820 - accuracy: 0.9367 - val_loss: 0.1642 - val_accuracy: 0.9450
Epoch 6/10
150/150 [==============================] - 5s 35ms/step - loss: 0.1307 - accuracy: 0.9600 - val_loss: 0.1257 - val_accuracy: 0.9567
Epoch 7/10
150/150 [==============================] - 6s 37ms/step - loss: 0.0895 - accuracy: 0.9704 - val_loss: 0.1188 - val_accuracy: 0.9600

In [54]:
model.save('braintumor.h5')

In [72]:
img= cv2.imread(r'D:\abcdefghj\Projects\Covid_Detection\Brain_Tumor\pred\pred39.jpg')
img= Image.fromarray(img)
# print(img)
img=img.resize((64,64))
img=np.array(img)
input=np.expand_dims(img, axis=0)

result= model.predict(input)
print(result)

1/1 [==============================] - 0s 15ms/step
[[1.]]


In [3]:
from tensorflow.keras.models import load_model
model1=load_model('braintumor.h5')


In [8]:
img= cv2.imread(r'C:\Users\dnyan\Downloads\R.png')
img= Image.fromarray(img)
# print(img)
img=img.resize((64,64))
img=np.array(img)
input=np.expand_dims(img, axis=0)

result= model1.predict(input)
print(result)

if result==1:
    print("tumor")

1/1 [==============================] - 0s 159ms/step
[[1.]]
tumor
